# 네이버 검색결과 이메일로 발송.

1. 전체 검색 결과를 구분별로 가져오기
2. 세부 파트별로 가져오기 (블로그, 뉴스, 카페 글)

## 1. 전체 검색 결과를 구분별로 가져오기
### 설계
1. 브라우저 설정 
2. target_url(https://search.naver.com/search.naver?query={검색내용}) 설정 및 접속
3. HTML 분석 & 내용을 형식별로 구분 (카페, 블로그, 지식In 등)
4. 가져올 내용들을 정리 (작성일자, 이미지, description, 제목, 링크, <블로그 이름>, 링크 url 등)
5. 시각화 (엑셀, or html) 등

#### 1. 브라우저 설정

In [2]:
# 1. 브라우저 설정
"""
정말 많은 brower가 있습니다만,
본 예제에서는 OS에 독립적으로 실행되길 원하므로,
보편적인 Chrome을 사용하겠습니다.
만약 chrome driver가 없다면 아래 링크에서 설치하십시요.
- http://chromedriver.storage.googleapis.com/index.html
"""

from selenium import webdriver

# TODO: Option에 대한 설명 필요.
# options = None
options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
    "download.default_directory": '.'
})
driver = webdriver.Chrome(options=options)
driver.maximize_window()
driver.implicitly_wait(10)


### 아래와 같은 많은 브라우저들이 있습니다.

# driver = webdriver.FireFox
# webdriver.FirefoxProfile

# driver = webdriver.Ie
# driver = webdriver.Opera

# driver = webdriver.PhantomJS

# driver = webdriver.Remote
# webdriver.DesiredCapabilities

# driver = webdriver.safairi

## ...


#### 2. target_url(https://search.naver.com/search.naver?query={검색내용}) 설정 및 접속

In [3]:
base_url = 'https://search.naver.com/search.naver?query='
search_content = '마약베개'

target_url = base_url + search_content

driver.get(target_url)

#### 3. HTML 분석 (Top-Down 방식) & 내용을 형식별로 구분 (카페, 블로그, 지식In 등)

In [4]:
## 검색 결과가 나오는 컨텐츠 
## <div id="main_pack" class="main_pack">...</div>
main_pack = driver.find_element_by_xpath('//*[@id="main_pack"]')
print(main_pack)
print(dir(main_pack))

## 연관검색어
related_keywords = main_pack.find_element_by_xpath('//*[@id="nx_related_keywords"]/dl/dd[1]/ul')

## 브랜드 서치 섹션
# naver는 동적으로 배치가 바뀝니다. 그러므로 식별자(class, id) 없이 무작정 xpath로 path를 복사하면,
# 다른 파일에서 동작을 하지 않을 수가 있습니다.
# 해당 예제에서는 class를 식별자로 하여 데이터를 가져오기로 합니다.
brand_search_section = main_pack.find_element_by_xpath('//*[@class="brand_search section"]')

## 지식쇼핑 섹션
sp_shop_section = main_pack.find_element_by_class_name('sp_shop_default')  # 이렇게 class로도 찾을 수 있습니다.

## 블로그 섹션
blog_section = main_pack.find_element_by_class_name('blog')

## cafe 섹션 
cafe_section = main_pack.find_element_by_class_name('cafe')



<selenium.webdriver.remote.webelement.WebElement (session="fceab85fc5b8788b88c6f52e9e8c8552", element="0.7448673880526091-1")>
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_execute', '_id', '_parent', '_upload', '_w3c', 'clear', 'click', 'find_element', 'find_element_by_class_name', 'find_element_by_css_selector', 'find_element_by_id', 'find_element_by_link_text', 'find_element_by_name', 'find_element_by_partial_link_text', 'find_element_by_tag_name', 'find_element_by_xpath', 'find_elements', 'find_elements_by_class_name', 'find_elements_by_css_selector', 'find_elements_by_id', 'find_elements_by_link_text', 'find_elements_by_name', 'find_elements_by_partial_link_text', 'find_elements_by_tag

#### 4. 가져올 내용들을 정리 (작성일자, 이미지, description, 제목, 링크, <블로그 이름>, 링크 url 등)

In [7]:
## 연관검색어 형식 구분
# keyword 별로 나열.
keywords = []
# print(dir(related_keywords))

keywords_tag = related_keywords.find_elements_by_tag_name('li')
for x in keywords_tag:
    keywords.append({
        'text':x.text,
        'link':x.find_element_by_tag_name('a').get_attribute('href')
    })
# pprint(keywords)

## 브랜드 서치 섹션 형식 구분
brand_search_section.screenshot('brand_search_section.png') # 해당부분만 스크린샷 찍기

## 지식쇼핑 섹션
sp_shop_section.screenshot('sp_shop.png')
print("C")

## 블로그 섹션
blog_list = []
blog_contents = blog_section.find_element_by_tag_name('ul').find_elements_by_tag_name('li')
for blog_content in blog_contents:
    title = blog_content.find_element_by_class_name('sh_blog_title')
    blog_list.append({
        'image':blog_content.find_element_by_class_name('thumb').find_element_by_tag_name('img').get_attribute('src'),
        'link':title.get_attribute('href'),
        'title':title.text,
        'desc':blog_content.find_element_by_class_name('sh_blog_passage').text
    })
# pprint(blog_list)
print("Hi")

## cafe 섹션
# 자세히 보면 블로그와 구조가 똑같아보여요!
cafe_list = []
cafe_contents = cafe_section.find_element_by_tag_name('ul').find_elements_by_tag_name('li')
for cafe_content in cafe_contents:
    title = cafe_content.find_element_by_class_name('sh_cafe_title')
    cafe_list.append({
        'image':cafe_content.find_element_by_class_name('thumb').find_element_by_tag_name('img').get_attribute('src'),
        'link':title.get_attribute('href'),
        'title':title.text,
        'desc':cafe_content.find_element_by_class_name('sh_cafe_passage').text
    })
pprint(cafe_list)
print("HO")

C
Hi


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"class name","selector":"thumb"}
  (Session info: chrome=72.0.3626.121)
  (Driver info: chromedriver=2.45.615355 (d5698f682d8b2742017df6c81e0bd8e6a3063189),platform=Mac OS X 10.13.5 x86_64)


In [ ]:
## 1. 전체 검색 결과를 구분별로 가져오기
### 설계
1. 브라우저 설정 
2. target_url(https://search.naver.com/search.naver?query={검색내용}) 설정 및 접속
3. HTML 분석 & 내용을 형식별로 구분 (카페, 블로그, 지식In 등)
4. 가져올 내용들을 정리 (작성일자, 이미지, description, 제목, 링크, <블로그 이름>, 링크 url 등)
5. 시각화 (엑셀, or html) 등